In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import geohash_hilbert as ghh

In [2]:
df = pd.read_csv('data/yellow_tripdata_2015_short.csv', index_col=0)
print ("Shape of data\n{}".format(df.shape))
df.head()

Shape of data
(770806, 18)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
61599709,1,2015-05-12 17:46:38,2015-05-12 18:18:09,1,9.10,-73.863525,40.770012,1,N,-73.982628,40.736942,1,30.0,1.0,0.5,7.45,5.54,44.79
24871662,1,2015-02-27 06:03:22,2015-02-27 06:09:52,1,2.60,-74.008308,40.714573,1,N,-73.988518,40.748821,1,9.0,0.5,0.5,1.50,0.00,11.80
19662635,2,2015-02-15 11:23:13,2015-02-15 11:29:37,1,1.28,-73.956635,40.775738,1,N,-73.967468,40.761639,2,7.0,0.0,0.5,0.00,0.00,7.80
15273088,2,2015-02-05 18:36:07,2015-02-05 18:36:40,1,0.00,-73.974190,40.754951,5,N,-73.974167,40.754822,1,52.0,0.0,0.5,11.47,5.33,69.60
67295899,2,2015-06-06 22:18:20,2015-06-06 22:40:32,3,4.55,-73.973450,40.784496,1,N,-73.993279,40.732471,2,17.5,0.5,0.5,0.00,0.00,18.80


In [3]:
if DEVELOPE:
    df = df.sample(frac=0.01)
else:
    pass
print ("Shape of data\n{}".format(df.shape))

Shape of data
(7708, 18)


In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'],
      dtype='object')

In [5]:
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
0,2,2015-04-12 12:40:45,2015-04-12 12:54:59,5,3.03,-73.992943,40.724190,1,N,-73.965622,40.762901,1,12.5,0.0,0.5,1.0,0.0,14.3
1,1,2015-02-12 05:48:42,2015-02-12 05:52:29,1,0.90,-73.998695,40.739899,1,N,-73.994865,40.750172,1,5.0,0.5,0.5,1.0,0.0,7.3
2,2,2015-05-29 14:20:45,2015-05-29 14:34:46,1,1.78,-73.965836,40.774090,1,N,-73.975861,40.788280,2,10.5,0.0,0.5,0.0,0.0,11.3
3,2,2015-03-02 11:48:38,2015-03-02 11:56:24,4,1.19,-73.969688,40.757206,1,N,-73.956558,40.767086,2,7.0,0.0,0.5,0.0,0.0,7.8
4,2,2015-06-11 07:34:54,2015-06-11 07:44:19,1,2.03,-73.969955,40.784592,1,N,-73.969551,40.762943,1,9.0,0.0,0.5,3.0,0.0,12.8


In [8]:
#add a column of geohash to dataframe
geo = {}
for i in range(len(df)):
    value = ghh.encode(df.pickup_longitude.iloc[i], df.pickup_latitude.iloc[i], precision=5)
    geo[i] = value
s = pd.Series(geo, name='geohash')
geo = pd.DataFrame(s)
df = df.merge(geo, left_index=True, right_index=True)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,geohash
0,2,2015-04-12 12:40:45,2015-04-12 12:54:59,5,3.03,-73.992943,40.724190,1,N,-73.965622,40.762901,1,12.5,0.0,0.5,1.0,0.0,14.3,SHG2O
1,1,2015-02-12 05:48:42,2015-02-12 05:52:29,1,0.90,-73.998695,40.739899,1,N,-73.994865,40.750172,1,5.0,0.5,0.5,1.0,0.0,7.3,SHG1e
2,2,2015-05-29 14:20:45,2015-05-29 14:34:46,1,1.78,-73.965836,40.774090,1,N,-73.975861,40.788280,2,10.5,0.0,0.5,0.0,0.0,11.3,SHG1G
3,2,2015-03-02 11:48:38,2015-03-02 11:56:24,4,1.19,-73.969688,40.757206,1,N,-73.956558,40.767086,2,7.0,0.0,0.5,0.0,0.0,7.8,SHG1V
4,2,2015-06-11 07:34:54,2015-06-11 07:44:19,1,2.03,-73.969955,40.784592,1,N,-73.969551,40.762943,1,9.0,0.0,0.5,3.0,0.0,12.8,SHBJ5


In [9]:
df.geohash.unique()

array(['SHG2O', 'SHG1e', 'SHG1G', 'SHG1V', 'SHBJ5', 'SHG1_', 'SHG1h',
       'SHGCe', 'SHG1Q', 'SHG1R', 'SHG1I', 'SHG7z', 'SHG1M', 'SHG1A',
       'SHG1S', 'SHG1b', 'SHFkj', 'SHGDK', 'SHGCa', 'SHG18', 'SHG2K',
       'SHG09', 'SHG06', 'SHG7r', 'SHG1B', 'SHG1T', 'SHG1O', 'SHBJ@',
       'SHG1a', 'SHGCO', 'SHG1f', 'SHG2L', 'SHG1H', 'SHG1d', 'SHG1U',
       'SHG0t', 'SHG1Y', 'SHG1c', 'SHG2U', 'SHG1X', 'SHG2M', 'SHBJr',
       'SHG1g', 'SHG2N', 'SHBJl', 'SHG1Z', 'SHBJt', 'SHG1p', 'V0000',
       'SHBJY', 'SHBJ4', 'SHG2S', 'SHFkk', 'SHFlA', 'SHG2J', 'SHGCN',
       'SHG15', 'SHBJ9', 'SHG1N', 'SHBJV', 'SHGCL', 'SHG3Z', 'SHG1E',
       'SHG1W', 'SHG1P', 'SHGCd', 'SHG1l', 'SHG7D', 'SHG1C', 'SHG1@',
       'SHBJ7', 'SHG2H', 'SHG7w', 'SHG2A', 'SHGC_', 'SHG2f', 'SHG1F',
       'SHGDL', 'SHG2p', 'SHBJi', 'SHBJR', 'SHBJu', 'SHG1J', 'SHG2Q',
       'SHG2I', 'SHGCQ', 'SHBJv', 'SHG0s', 'SHG7m', 'SHBJU', 'SHG1t',
       'SHG84', 'SHG2Z', 'SHFkn', 'SHFl6', 'SHG2P', 'SHBJm', 'SHG1D',
       'SHG7_', 'SHG